In [204]:
from bs4 import BeautifulSoup as bs
from splinter import Browser
import time
import pandas as pd
import requests

In [211]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

## NASA Mars News

In [206]:
nasa_url = 'https://mars.nasa.gov/news/'
browser.visit(nasa_url)
time.sleep(1)

nasa_soup = bs(browser.html,'html.parser')

In [207]:
stories = nasa_soup.body.find('ul', class_='item_list')

first_story = stories.find('li', class_='slide')

story_title = f"News: {first_story.find('div', class_='content_title').text}"
story_text = first_story.find('div', class_='article_teaser_body').text

print(f"{story_title}: {story_text}")

Curiosity on the Move Again: NASA's Mars Curiosity rover drove about 197 feet over the weekend to a site called Lake Orcadie, pushing its total odometry to over 12 miles.


## JPL Mars Space Images

In [209]:
jpl_url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(jpl_url)

jpl_soup = bs(browser.html, 'html.parser')

In [191]:
featured_image = jpl_soup.find('article', class_='carousel_item')

In [210]:
featured_image_style = featured_image['style']

featured_image_url = str.replace(featured_image_style,"background-image: url('",'')
featured_image_url = str.replace(featured_image_url,"');",'')

featured_image_url = str.replace(jpl_url,'/spaceimages/?search=&category=Mars',featured_image_url)
print(featured_image_url)

https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA20464-1920x1200.jpg


In [236]:
featured_image_alt = featured_image['alt']
print(featured_image_alt)

Pluto's Mysterious, Floating Hills


## Mars Weather

In [212]:
weather_url = 'https://twitter.com/marswxreport?lang=en'
browser.visit(weather_url)

weather_soup = bs(browser.html,'html.parser')

In [213]:
timeline = weather_soup.body.find('div', class_='ProfileTimeline')

latest_tweet = timeline.find('li', class_='js-stream-item')

mars_weather = latest_tweet.find('p', class_='tweet-text').text
print(mars_weather)

Sol 2224 (2018-11-08), high 0C/32F, low -72C/-97F, pressure at 8.65 hPa, daylight 06:19-18:36


## Mars Facts

In [232]:
facts_url = 'https://space-facts.com/mars/'

mars_table = pd.read_html(facts_url)[0]
mars_table

,0,1
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.42 x 10^23 kg (10.7% Earth)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.52 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-153 to 20 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [235]:
mars_facts = pd.DataFrame.to_html(mars_table, index=False, header=False)
print(mars_facts)

<table border="1" class="dataframe">
  <tbody>
    <tr>
      <td>Equatorial Diameter:</td>
      <td>6,792 km</td>
    </tr>
    <tr>
      <td>Polar Diameter:</td>
      <td>6,752 km</td>
    </tr>
    <tr>
      <td>Mass:</td>
      <td>6.42 x 10^23 kg (10.7% Earth)</td>
    </tr>
    <tr>
      <td>Moons:</td>
      <td>2 (Phobos &amp; Deimos)</td>
    </tr>
    <tr>
      <td>Orbit Distance:</td>
      <td>227,943,824 km (1.52 AU)</td>
    </tr>
    <tr>
      <td>Orbit Period:</td>
      <td>687 days (1.9 years)</td>
    </tr>
    <tr>
      <td>Surface Temperature:</td>
      <td>-153 to 20 °C</td>
    </tr>
    <tr>
      <td>First Record:</td>
      <td>2nd millennium BC</td>
    </tr>
    <tr>
      <td>Recorded By:</td>
      <td>Egyptian astronomers</td>
    </tr>
  </tbody>
</table>


## Mars Hemispheres

In [216]:
hemispheres_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(hemispheres_url)

hemispheres_soup = bs(browser.html,'html.parser')

In [217]:
results = hemispheres_soup.body.find('div', class_='result-list')

hemisphere_results = results.find_all('div', class_='item')

In [200]:
hemisphere_image_urls = []

for hemisphere in hemisphere_results:
    hemisphere_description = hemisphere.find('div',class_='description')
    
    hemisphere_name = hemisphere_description.find('h3').text
    hemisphere_name = str.replace(hemisphere_name,' Enhanced', '')
    
    hemisphere_link = hemisphere.find('a', class_='itemLink')['href']
    hemisphere_link = str.replace(url,'/search/results?q=hemisphere+enhanced&k1=target&v1=Mars',hemisphere_link)
    
    image_html = requests.get(hemisphere_link).text
    image_soup = bs(image_html,'html.parser')
    
    image = image_soup.find('div', class_='wide-image-wrapper').find('img', class_='wide-image')
    
    image_url = image['src']
    image_url = str.replace(hemispheres_url,'/search/results?q=hemisphere+enhanced&k1=target&v1=Mars', image_url)
    
    hemisphere_dict = {'title': hemisphere_name,
                      'image_url': image_url}
    
    hemisphere_image_urls.append(hemisphere_dict)

print(hemisphere_image_urls)

[{'title': 'Cerberus Hemisphere', 'image_url': 'https://astrogeology.usgs.gov/cache/images/cfa62af2557222a02478f1fcd781d445_cerberus_enhanced.tif_full.jpg'}, {'title': 'Schiaparelli Hemisphere', 'image_url': 'https://astrogeology.usgs.gov/cache/images/3cdd1cbf5e0813bba925c9030d13b62e_schiaparelli_enhanced.tif_full.jpg'}, {'title': 'Syrtis Major Hemisphere', 'image_url': 'https://astrogeology.usgs.gov/cache/images/ae209b4e408bb6c3e67b6af38168cf28_syrtis_major_enhanced.tif_full.jpg'}, {'title': 'Valles Marineris Hemisphere', 'image_url': 'https://astrogeology.usgs.gov/cache/images/7cf2da4bf549ed01c17f206327be4db7_valles_marineris_enhanced.tif_full.jpg'}]
